<a href="https://colab.research.google.com/github/coder272377/dvp/blob/develop/tictop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tic Top : Top Five Equal Weight



In [ ]:
interval = "5m"
previous_days = 5
tickers = tuple("AAPL TSLA GOOG AMZN MSFT".split())



## Package management



In [ ]:
import datetime
import functools
import math

from typing import List, Tuple, Union



In [ ]:
Version = Union[List[int], Tuple]



In [ ]:
def check_version(req:Version, version_str:str):
  v0, v1, v2 = version_str.split('.')

  result = False

  if int(v0) > req[0]:
    result = True
  elif int(v1) > req[1]:
    result = True
  elif int(v2) >= req[2]:
    result = True

  return result



Test function `check_version()` before using



In [ ]:
assert check_version([0, 9, 9], "0.10.0")



In [ ]:
assert check_version([0, 10, 0], "0.10.0")



How do I unload (reload) a Python module? https://stackoverflow.com/questions/437589/how-do-i-unload-reload-a-python-module



In [ ]:
import pandas as pd
if not check_version([1, 3, 5], pd.__version__):
  !pip install --upgrade pandas
  import importlib
  importlib.reload(pd)
  assert check_version([1, 3, 5], pd.__version__)



Unable to get stock data from yahoo with pandas_datareader

https://stackoverflow.com/questions/68327968/unable-to-get-stock-data-from-yahoo-with-pandas-datareader/68435757#68435757

Download Financial Dataset Using Yahoo Finance in Python | A Complete Guide https://www.analyticsvidhya.com/blog/2021/06/download-financial-dataset-using-yahoo-finance-in-python-a-complete-guide/



yfinance Library – A Complete Guide https://algotrading101.com/learn/yfinance-guide/


In [ ]:
import pandas as pd



In [ ]:
try:
  import yfinance as yf
except ModuleNotFoundError:
  !pip install yfinance
  !pip install yahoofinancials
  import yfinance as yf



Charting Candlestick_OHLC one minute bars with Pandas and Matplotlib https://stackoverflow.com/questions/41821916/charting-candlestick-ohlc-one-minute-bars-with-pandas-and-matplotlib

Candlestick Chart in Python (mplfinance, plotly, bokeh, bqplot and cufflinks) https://coderzcolumn.com/tutorials/data-science/candlestick-chart-in-python-mplfinance-plotly-bokeh#1

In [ ]:
try:
  import mplfinance as fplt
except ModuleNotFoundError:
  !pip install mplfinance
  import mplfinance as fplt



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss
import scipy.signal as signal



## Dates



In [ ]:
@functools.lru_cache()
def tz_ny() -> str:
  return "America/New_York"



In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
@functools.lru_cache()
def USBday():
  return pd.tseries.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())



In [ ]:
def now() -> pd.Timestamp:
  return pd.Timestamp.now(tz=tz_ny())



In [ ]:
@functools.lru_cache()
def get_start_date(n_days:int, end_date=now(), unit=USBday()):
  return end_date + unit * (-n_days)



In [ ]:
def test_get_start_date__default_argument():
  result = get_start_date(5)
  assert result < now(), (result, now)



In [ ]:
test_get_start_date__default_argument()



In [ ]:
def get_nyse_open(t:pd.Timestamp, tz=tz_ny()):
  """
  09:30 of given timestamp in NY time zone
  """
  return get_nyse_time(t, hour=9, minute=30, tz=tz)



In [ ]:
def get_nyse_close(t:pd.Timestamp, tz=tz_ny()):
  """
  16:00 of given timestamp in NY time zone
  """
  return get_nyse_time(t, hour=16, minute=00, tz=tz)



In [ ]:
@functools.lru_cache()
def get_nyse_time(t:pd.Timestamp, hour:int, minute:int, tz=tz_ny()):
  """
  New timestamp with given hour & minute in NY time zone
  """
  return pd.Timestamp(
    year=t.year, month=t.month, day=t.day, hour=hour, minute=minute, tz=tz,
  )



## Data-getters



In [ ]:
def get_df_start_end(ticker:str, start:pd.Timestamp, end:pd.Timestamp=now(), interval:str=interval):
  assert start < end
  if interval[-1] == 'm':
    delta = now() - start
    assert (pd.Timedelta(60, 'D') > delta), f"{start} older than 60 days for interval {interval!r}"
  tic = yf.Ticker(ticker)
  return tic.history(start=start, end=end, interval=interval)



In [ ]:
df = get_df_start_end("SPY", pd.Timestamp("2022-01-20", tz=tz_ny()), interval="5m")
df.tail()



In [ ]:
def get_df(ticker:str, n_days:int, end_date:pd.Timestamp=now(), interval:str=interval):
  return get_df_start_end(ticker, get_start_date(n_days, end_date=end_date), end=end_date, interval=interval)



In [ ]:
def get_days_df(ticker:str, n_days:int=previous_days, end_date:pd.Timestamp=now(), interval:str="1D") -> pd.DataFrame:
  return get_df(ticker, n_days, end_date, interval)



In [ ]:
def get_minutes_df(ticker:str, n_days:int=2, end_date:pd.Timestamp=now(), interval:str=interval) -> pd.DataFrame:
  return get_df(ticker, n_days, end_date, interval)



Test daily data getter functions



In [ ]:
def test_get_days_df_sunday():
  tic = "GOOG"
  days = 5
  end_date = pd.Timestamp("2022-01-23 10:50")
  df_days = get_days_df(tic, n_days=days, end_date=end_date, interval="1D")

  assert df_days.shape[0], df_days.shape
  assert pd.Timestamp("2022-01-21 00:00:00") == df_days.index[-1]



In [ ]:
test_get_days_df_sunday()



In [ ]:
def test_get_days_df__holiday():
  tic = "MSFT"
  days = 5
  end_date = pd.Timestamp("2021-06-02 09:50", tz=tz_ny())
  df_days = get_days_df(tic, n_days=days, end_date=end_date, interval="1D")

  assert pd.Timestamp("2021-06-01 00:00:00") == df_days.index[-1], df_days



In [ ]:
test_get_days_df__holiday()



Test minute data getter functions



In [ ]:
def test_get_minutes_df__today():
  tic = "GOOG"
  days = 2
  end_date = now()
  df_minutes = get_minutes_df(tic, n_days=days, end_date=end_date, interval="5m")

  assert pd.Timedelta(20, 'h') < (df_minutes.index[-1] - df_minutes.index[0]), df_minutes



In [ ]:
test_get_minutes_df__today()



In [ ]:
def get_prev_index(df:pd.DataFrame):
  """
  Get indices before the second last day market close
  """

  fin = df.index[-1]
  fin_begin = get_nyse_time(fin, hour=0, minute=0)
  prev = df.index < fin_begin

  index_before_fin = df.index[prev]
  prev_close = get_nyse_close(index_before_fin[-1])

  return index_before_fin[index_before_fin < prev_close]



In [ ]:
def get_last_day_df(df:pd.DataFrame):
  index_prev = get_prev_index(df)
  return df.drop(index_prev)



## Percentage-calculator



In [ ]:
def calc_pct(tic:str):
  df_minutes = get_minutes_df(tic)
  assert df_minutes.shape[0], df_minutes.shape

  index_prev = get_prev_index(df_minutes)

  previous_close = df_minutes.at[index_prev[-1], "Close"]

  df_current_date = get_last_day_df(df_minutes)

  ser_current_date = df_current_date["Close"]
  assert index_prev[-1] < ser_current_date.index[0]

  return ser_current_date.mul(100.0 / previous_close) - 100.0



In [ ]:
def ave_pct(tickers:Tuple[str]=tickers):

  df_pct = pd.DataFrame(
      {tic:calc_pct(tic) for tic in tickers},
      columns=tickers
  )

  result = df_pct.mean(axis=1)

  return result



## Chart-plotter



In [ ]:
def get_close_upper_lower(close:pd.Series) -> Tuple[float]:
  """
  Estimate lower & upper limits from close prices
  """
  close_max = close.max()
  close_min = close.min()

  delta_close = close_max - close_min

  close_lower = close_min - delta_close * 0.1
  close_upper = close_max + delta_close * 0.1

  return close_lower, close_upper


In [ ]:
def plot_candlestick(df:pd.DataFrame, ax=None):
  """
  Candle stick chart of a given dataframe
  """
  if ax is None:
    fig, ax = plt.subplots(figsize=(16, 9))

  close_lower, close_upper = get_close_upper_lower(df["Close"])

  fplt.plot(
      df,
      type='candle',
      style='charles',
      ylabel='Price ($)',
      ax=ax,
      datetime_format="%y/%m/%d %H:%M"
  );

  ax.grid(True)

  ylim = ax.get_ylim()

  new_ylim = (
      max(ylim[0], close_lower),
      min(ylim[1], close_upper),
  )
  ax.set_ylim(new_ylim)

  return ax



In [ ]:
def plot_candle_pct(tic, figsize:Tuple[int]=(24,10)):
  fig, ax = plt.subplots(2, 1, figsize=figsize)
  # fig.subplots_adjust(hspace=0)

  df_minutes = get_minutes_df(tic)

  fig.suptitle(f"{tic} {df_minutes.index[-1].date()}")

  plot_candlestick(
      get_last_day_df(df_minutes),
      ax=ax[0]
  )

  xlim = ax[0].get_xlim()
  xticks = ax[0].get_xticks()
  xticklabels = ax[0].get_xticklabels()
  # xmajorticklabels = ax[0].get_xmajorticklabels()

  df_pct = ave_pct()
  df_pct.plot.bar(
      ax=ax[1], grid=True, ylabel="%",
      xticks=xticks,
  )
  ax[1].set_xticklabels(xticklabels)
  ax[1].set_xlim(xlim);



In [ ]:
print(now())
get_last_day_df(get_minutes_df("SPY"))



In [ ]:
plot_candle_pct("SPY")

